## Pytorch LSTM auto convert test 

### 1.Build LSTM

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import onnx
import onnxruntime

In [2]:
class LSTMModel(nn.Module):

    def __init__(self, window_size, input_size):
        super().__init__()

        self.window_size = window_size
        self.input_size = input_size
        self.LSTM = nn.LSTM(input_size=input_size, hidden_size=window_size, batch_first=True, bidirectional=True)
        self.linear_out = nn.Linear(window_size * 2, 2)

    def forward(self, x):
        output, (hn, cn) = self.LSTM(x)
        out = self.linear_out(output[-1][-1])

        return out

In [3]:
window_size = 60
input_size = 9
model = LSTMModel(window_size, input_size)
model.eval()

LSTMModel(
  (LSTM): LSTM(9, 60, batch_first=True, bidirectional=True)
  (linear_out): Linear(in_features=120, out_features=2, bias=True)
)

### 2.Convert model to ONNX type

ONNX conversion requires one data to help fix the graph, and also fix BatchSize.

Here we use one random data to do this.

In [4]:
batch_size = 1
x = torch.randn(batch_size, 60, 9, requires_grad=True)
torch_out = model(x)

In [5]:
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "LSTM.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # wether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'] # the model's output names
                  )
#torch.onnx.export(model, dummy_input, "alexnet.onnx") #simple version

In [6]:
onnx_model = onnx.load("LSTM.onnx")
onnx.checker.check_model(onnx_model)

print(onnx.helper.printable_graph(onnx_model.graph))

graph torch-jit-export (
  %input[FLOAT, 1x60x9]
) optional inputs with matching initializers (
  %linear_out.bias[FLOAT, 2]
  %152[INT64, 1]
  %153[INT64, 1]
  %194[FLOAT, 2x240x9]
  %195[FLOAT, 2x240x60]
  %196[FLOAT, 2x480]
  %197[FLOAT, 120x2]
) {
  %11 = Constant[value = <Scalar Tensor []>]()
  %12 = Shape(%input)
  %13 = Gather[axis = 0](%12, %11)
  %17 = Unsqueeze[axes = [0]](%13)
  %19 = Concat[axis = 0](%152, %17, %153)
  %20 = ConstantOfShape[value = <Tensor>](%19)
  %21 = Transpose[perm = [1, 0, 2]](%input)
  %138, %139, %140 = LSTM[direction = 'bidirectional', hidden_size = 60](%21, %194, %195, %196, %, %20, %20)
  %141 = Transpose[perm = [0, 2, 1, 3]](%138)
  %142 = Constant[value = <Tensor>]()
  %143 = Reshape(%141, %142)
  %144 = Transpose[perm = [1, 0, 2]](%143)
  %145 = Constant[value = <Scalar Tensor []>]()
  %146 = Gather[axis = 0](%144, %145)
  %147 = Constant[value = <Scalar Tensor []>]()
  %148 = Gather[axis = 0](%146, %147)
  %150 = MatMul(%148, %197)
  %output =

In [7]:
ort_session = onnxruntime.InferenceSession("LSTM.onnx")

input_name = ort_session.get_inputs()[0].name
print("Input name  :", input_name)
input_shape = ort_session.get_inputs()[0].shape
print("Input shape :", input_shape)
input_type = ort_session.get_inputs()[0].type
print("Input type  :", input_type)

output_name = ort_session.get_outputs()[0].name
print("Output name  :", output_name)  
output_shape = ort_session.get_outputs()[0].shape
print("Output shape :", output_shape)
output_type = ort_session.get_outputs()[0].type
print("Output type  :", output_type)


Input name  : input
Input shape : [1, 60, 9]
Input type  : tensor(float)
Output name  : output
Output shape : [2]
Output type  : tensor(float)


### 3.Running Check

Because Network still Not trained, So only use Random number to see if it can work.

In [8]:
xt = np.random.random(input_shape)
xt = xt.astype(np.float32)

In [9]:
%%time
result = ort_session.run([output_name], {input_name: xt})

RuntimeError: Method run failed due to: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Non-zero status code returned while running Node:  Status Message: indices element out of data bounds, idx=-1 data_dim=1

### Error reason:
After search on Github find that because ONNX Runtime version problem:

Now newest pipy version is 0.5.0 released on 02-aug

Bug already fixed in source file.

Need waiting for new releas of onnxruntime.

### 4.Consistency check

Reserved Consistency Detection

In the future, the trained models can use this for transformed consistency checking.

In [ ]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
x_in = to_numpy(x)
print(x_in.shape)
ort_inputs = {ort_session.get_inputs()[0].name: x_in }
ort_outs = ort_session.run([ort_session.get_outputs()[0].name], ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")